In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import cv2
import pickle
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import xgboost as xgb

In [15]:
#Resizing images is optional, CNNs are ok with large images
SIZE_X = 501 #Resize images (height  = X, width = Y)
SIZE_Y = 501


#Capture training image info as a list
train_images = []

dirname = './dataset/data'  
for fname in os.listdir(dirname):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)   
 
    img = cv2.resize(img, (SIZE_Y, SIZE_X))
    train_stack = np.stack((img,)*3, axis=-1)
    train_images.append(train_stack)
    #train_labels.append(label)
#Convert list to array for machine learning processing        
dataimages = np.array(train_images)


#Capture mask/label info as a list
train_masks = [] 
dirname = './dataset/labels'
for fname in os.listdir(dirname):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)       
    img = cv2.resize(img, (SIZE_Y, SIZE_X))
    #mask_stack = np.stack((img,)*3, axis=-1)
    train_masks.append(img)
    #print(f"Unique vlaues in labels image {np.unique(img)}")
        #train_labels.append(label)
#Convert list to array for machine learning processing          
labelimages = np.array(train_masks)
#Cant change it here as I have the features from the VGG in the other Script
X_train, X_test, y_train, y_test = train_test_split(dataimages, labelimages, test_size=0.95, random_state=42)


#Use customary x_train and y_train variables

y_train = np.expand_dims(y_train, axis=3) #May not be necessary.. leftover from previous code 
y_test = np.expand_dims(y_test, axis=3)

In [16]:
#Import the feature Data from a file that I have written it to 
dataset = pd.read_feather('FeatureData.feather')
#Get Test Train Data
# This is so that we dont overwork our Ram
dataset = dataset.sample(frac=0.005, random_state=1)


#Redefine X and Y for Random Forest
X_for_training = dataset.drop(labels = ['Label'], axis=1)
X_for_training = X_for_training.values  #Convert to array
Y_for_training = dataset['Label']
Y_for_training = Y_for_training.values  #Convert to array
mapping = {0: 0, 128: 1, 255: 2}

# Vectorized mapping
Y_for_training = np.vectorize(mapping.get)(Y_for_training)

In [17]:
#Load the Boost Model
filename = 'model_XG.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [18]:

# Neural Network

# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert the data to PyTorch tensors and move them to the selected device
X_train_tensor = torch.tensor(X_for_training).float().to(device)
Y_train_tensor = torch.tensor(Y_for_training).long().to(device)

# Create a Dataset and a DataLoader
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(64, 128)  # Adjust the input dimensions if needed
        self.fc2 = nn.Linear(128, 3)   # Adjust the output dimensions based on the number of classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model and move it to the selected device
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
model.train()  # Set the model to training mode
best_loss = float('inf')  # Initialize best loss to a large value

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the same device as the model
        inputs = inputs.view(inputs.size(0), -1)  # Flatten the input features
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print and save best model
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), 'best_model_NN.pth')
        print(f'New best model saved with loss: {best_loss:.4f}')
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")


New best model saved with loss: 1.9666
Epoch [1/100], Loss: 1.9666
New best model saved with loss: 0.9300
Epoch [2/100], Loss: 0.9300
Epoch [3/100], Loss: 0.9843
New best model saved with loss: 0.7471
Epoch [4/100], Loss: 0.7471
New best model saved with loss: 0.5740
Epoch [5/100], Loss: 0.5740
New best model saved with loss: 0.5705
Epoch [6/100], Loss: 0.5705
Epoch [7/100], Loss: 1.0469
Epoch [8/100], Loss: 0.6151
Epoch [9/100], Loss: 0.9585
New best model saved with loss: 0.5323
Epoch [10/100], Loss: 0.5323
New best model saved with loss: 0.4428
Epoch [11/100], Loss: 0.4428
Epoch [12/100], Loss: 0.6773
New best model saved with loss: 0.3780
Epoch [13/100], Loss: 0.3780
Epoch [14/100], Loss: 0.3907
Epoch [15/100], Loss: 0.3796
New best model saved with loss: 0.3322
Epoch [16/100], Loss: 0.3322
Epoch [17/100], Loss: 0.3548
New best model saved with loss: 0.2675
Epoch [18/100], Loss: 0.2675
New best model saved with loss: 0.2630
Epoch [19/100], Loss: 0.2630
New best model saved with los

In [22]:
#Load the model 
# Initialize the model
loaded_model_NN = SimpleNN()

# Load the saved state dictionary
loaded_model_NN.load_state_dict(torch.load('best_model_NN.pth'))

# Move the model to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = loaded_model_NN.to(device)
loaded_model.eval()  # Set the model to evaluation mode

SimpleNN(
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
)

In [25]:
# Assuming X_test is your test data
# Convert the test data to a PyTorch tensor

first_image_batch = np.expand_dims(X_test[0], axis=0)  # Reshape to (1, 501, 501, 3)
X_test_feature = new_model.predict(first_image_batch)
X_test_feature = X_test_feature.reshape(-1, X_test_feature.shape[3])


X_test_feature_tensor = torch.tensor(X_test_feature).float()

# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model and data to the device
loaded_model_NN = loaded_model_NN.to(device)
X_test_feature_tensor = X_test_feature_tensor.to(device)

# Make sure the model is in evaluation mode
loaded_model_NN.eval()

# Perform the prediction
with torch.no_grad():  # Disable gradient computation for inference
    first_image_batch = X_test_feature_tensor.unsqueeze(0)  # Add a batch dimension
    outputs = loaded_model_NN(first_image_batch)

# Convert outputs to probabilities using softmax (if your model doesn't already include it)
probabilities = torch.nn.functional.softmax(outputs, dim=1)

# Convert probabilities to a pandas DataFrame
class_probabilities_df = pd.DataFrame(probabilities.cpu().numpy(), 
                                      columns=[f'Class_{i+1}_Prob' for i in range(probabilities.shape[1])])

filtered_df = class_probabilities_df[(class_probabilities_df < 0.6).all(axis=1)]

# Print the DataFrame
print(filtered_df.to_string(index=False))

# Convert the model output to a prediction
_, predicted = torch.max(outputs.data, 1)
prediction_image = predicted.cpu().numpy().reshape((501, 501))

# Rest of your visualization code
original_image = np.squeeze(X_test[0])
labeled_images = np.squeeze(y_test[0])

plt.figure(figsize=(15, 5))

# Original Image
plt.subplot(1, 3, 1)
plt.imshow(original_image, cmap='gray')
plt.title('Original Image')

# Labeled Image
plt.subplot(1, 3, 2)
plt.imshow(labeled_images, cmap='gray')
plt.title('Labeled Image')

# Prediction Image
plt.subplot(1, 3, 3)
plt.imshow(prediction_image, cmap='gray')
plt.title('Prediction')

plt.show()

NameError: name 'new_model' is not defined